# 07a 早期经典 CNN 架构：LeNet、AlexNet、VGGNet

> 从手写识别到 ImageNet 霸主：CNN 发展的奠基之路

---

**本章重点**：理解 CNN 从诞生到成熟的演进历程，掌握三个里程碑架构的核心设计思想。

## 📚 学习目标

- [ ] 理解 CNN 发展的历史脉络
- [ ] **掌握 LeNet-5 的开创性设计**
- [ ] **理解 AlexNet 带来的关键创新（ReLU、Dropout、GPU）**
- [ ] **掌握 VGGNet 的 3×3 小卷积堆叠思想**
- [ ] 能够使用 PyTorch 实现这三个经典架构
- [ ] 理解架构演进背后的设计哲学

## 🔍 前置知识

在开始之前，请确保你已经掌握：

- **02_convolution_math.ipynb** - 卷积的数学原理
- **03_convolution_from_scratch.ipynb** - 卷积的从零实现
- **04_pooling_layers.ipynb** - 池化层原理

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

---

# CNN 发展时间线

```
1998: LeNet-5      → 第一个成功的 CNN，银行支票识别
        ↓
     （AI 寒冬，神经网络研究低迷）
        ↓
2012: AlexNet      → ReLU + Dropout + GPU，ImageNet 冠军，开启深度学习时代
        ↓
2014: VGGNet       → 3×3 小卷积堆叠，证明"更深更好"
        ↓
2014: GoogLeNet    → Inception 多尺度并行卷积
        ↓
2015: ResNet       → 残差连接，突破深度限制（152层）
```

In [ ]:
# 可视化 CNN 发展时间线

fig, ax = plt.subplots(figsize=(14, 6))

# 时间线数据
years = [1998, 2012, 2014, 2014, 2015, 2017, 2019, 2020]
names = ['LeNet-5', 'AlexNet', 'VGGNet', 'GoogLeNet', 'ResNet', 'DenseNet', 'EfficientNet', 'ViT']
depths = [5, 8, 19, 22, 152, 121, 'B7:66', 'L/16']
top1 = [99.2, 84.7, 92.7, 93.3, 96.4, 96.5, 97.1, 97.3]  # ImageNet 或 MNIST
colors = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6', '#f39c12', '#1abc9c', '#e67e22', '#34495e']
highlights = [True, True, True, False, False, False, False, False]  # 本章重点

# 绘制时间线
ax.axhline(y=0, color='gray', linewidth=2, alpha=0.5)

for i, (year, name, depth, acc, color, hl) in enumerate(zip(years, names, depths, top1, colors, highlights)):
    y_offset = 0.5 if i % 2 == 0 else -0.5
    
    # 圆点大小表示是否是本章重点
    size = 300 if hl else 150
    edgecolor = 'gold' if hl else 'white'
    linewidth = 3 if hl else 1
    
    ax.scatter(year, 0, s=size, c=color, zorder=5, edgecolor=edgecolor, linewidth=linewidth)
    ax.plot([year, year], [0, y_offset * 0.8], color=color, linewidth=2)
    
    # 标签
    label = f"{name}\n({depth} layers)" if isinstance(depth, int) else f"{name}\n({depth})"
    ax.annotate(label, (year, y_offset), ha='center', va='center' if y_offset > 0 else 'center',
                fontsize=10, fontweight='bold' if hl else 'normal',
                bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.3 if hl else 0.1))

# 标记本章重点
ax.axvspan(1996, 2015, alpha=0.1, color='yellow')
ax.text(2006.5, 0.9, '📖 本章重点', fontsize=12, ha='center', fontweight='bold')

ax.set_xlim(1995, 2022)
ax.set_ylim(-1.2, 1.2)
ax.set_xlabel('年份', fontsize=12)
ax.set_title('CNN 架构发展时间线', fontsize=14, fontweight='bold')
ax.set_yticks([])

plt.tight_layout()
plt.show()

print("💡 本章将深入学习三个黄色边框标记的架构：LeNet-5、AlexNet、VGGNet")
print("   这三个架构奠定了现代 CNN 的基础设计范式")

---

# Part 1: LeNet-5 (1998) - CNN 的开山之作

## 1.1 历史背景

**Yann LeCun** 于 1998 年发表的 LeNet-5 是第一个在商业上成功应用的卷积神经网络。

**应用场景**：美国银行支票上手写数字的自动识别

**历史意义**：
- 首次证明 CNN 可以在实际任务中超越传统方法
- 确立了「卷积层 + 池化层 + 全连接层」的基本架构模式
- 提出了参数共享和局部感受野的概念

## 1.2 LeNet-5 架构详解

```
输入 (1, 32, 32)  ← 灰度图像
        ↓
   C1: Conv 5×5, 6 filters    → (6, 28, 28)
        ↓
   S2: AvgPool 2×2            → (6, 14, 14)   ← 原论文叫 "Subsampling"
        ↓
   C3: Conv 5×5, 16 filters   → (16, 10, 10)
        ↓
   S4: AvgPool 2×2            → (16, 5, 5)
        ↓
   C5: Conv 5×5, 120 filters  → (120, 1, 1)   ← 实际上等价于全连接
        ↓
   F6: Fully Connected        → 84
        ↓
   Output                     → 10 (数字 0-9)
```

**注意**：
- 原论文使用 **Tanh** 作为激活函数（当时 ReLU 还没被提出）
- 使用 **Average Pooling**（不是 Max Pooling）
- 输入尺寸是 32×32（MNIST 是 28×28，需要 padding）

In [ ]:
# 可视化 LeNet-5 架构

fig, ax = plt.subplots(figsize=(16, 5))

# 定义每层的信息
layers = [
    {'name': 'Input', 'size': (32, 32), 'channels': 1, 'x': 0},
    {'name': 'C1\nConv 5×5', 'size': (28, 28), 'channels': 6, 'x': 1.5},
    {'name': 'S2\nPool 2×2', 'size': (14, 14), 'channels': 6, 'x': 3},
    {'name': 'C3\nConv 5×5', 'size': (10, 10), 'channels': 16, 'x': 4.5},
    {'name': 'S4\nPool 2×2', 'size': (5, 5), 'channels': 16, 'x': 6},
    {'name': 'C5\nConv 5×5', 'size': (1, 1), 'channels': 120, 'x': 7.5},
    {'name': 'F6\nFC', 'size': (1, 1), 'channels': 84, 'x': 9},
    {'name': 'Output\n10 classes', 'size': (1, 1), 'channels': 10, 'x': 10.5},
]

# 绘制每层
for layer in layers:
    x = layer['x']
    h = layer['size'][0] / 32 * 2  # 归一化高度
    w = 0.4
    c = layer['channels']
    
    # 用矩形表示特征图
    rect = plt.Rectangle((x - w/2, -h/2), w, h, 
                          facecolor='steelblue', edgecolor='black', alpha=0.7)
    ax.add_patch(rect)
    
    # 标签
    ax.text(x, -h/2 - 0.3, layer['name'], ha='center', va='top', fontsize=9)
    ax.text(x, h/2 + 0.1, f"{layer['size'][0]}×{layer['size'][1]}×{c}", 
            ha='center', va='bottom', fontsize=8, color='gray')

# 箭头连接
for i in range(len(layers) - 1):
    x1 = layers[i]['x'] + 0.2
    x2 = layers[i+1]['x'] - 0.2
    ax.annotate('', xy=(x2, 0), xytext=(x1, 0),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

ax.set_xlim(-1, 12)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('LeNet-5 架构图', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 1.3 PyTorch 实现 LeNet-5

In [ ]:
class LeNet5(nn.Module):
    """
    LeNet-5 经典实现 (Yann LeCun, 1998)
    
    架构特点:
        - 卷积层与池化层交替排列
        - 使用 Tanh 激活函数（原论文）
        - 使用 Average Pooling
        - 参数量约 60K
    
    输入:
        x: (batch, 1, 32, 32) 灰度图像
        如果输入是 28×28（MNIST），会自动 padding 到 32×32
    
    输出:
        logits: (batch, num_classes) 分类 logits
    """
    
    def __init__(self, num_classes=10, use_original_activation=True):
        """
        参数:
            num_classes: 分类数量，默认 10（数字 0-9）
            use_original_activation: 是否使用原论文的 Tanh，False 则使用 ReLU
        """
        super().__init__()
        
        # 选择激活函数
        self.activation = nn.Tanh() if use_original_activation else nn.ReLU()
        
        # ========================================
        # 特征提取部分
        # ========================================
        
        # C1: 第一个卷积层
        # 输入: (1, 32, 32) → 输出: (6, 28, 28)
        # 卷积核: 5×5, 无 padding, stride=1
        # 输出尺寸: (32 - 5 + 1) = 28
        self.conv1 = nn.Conv2d(
            in_channels=1,      # 灰度图像
            out_channels=6,     # 6 个卷积核
            kernel_size=5,      # 5×5
            stride=1,
            padding=0
        )
        
        # S2: 第一个池化层（原论文叫 Subsampling）
        # 输入: (6, 28, 28) → 输出: (6, 14, 14)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # C3: 第二个卷积层
        # 输入: (6, 14, 14) → 输出: (16, 10, 10)
        # 注意: 原论文中 C3 有复杂的连接模式，这里简化为全连接
        self.conv2 = nn.Conv2d(
            in_channels=6,
            out_channels=16,
            kernel_size=5
        )
        
        # S4: 第二个池化层
        # 输入: (16, 10, 10) → 输出: (16, 5, 5)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # C5: 第三个卷积层（实际上等价于全连接）
        # 输入: (16, 5, 5) → 输出: (120, 1, 1)
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=120,
            kernel_size=5
        )
        
        # ========================================
        # 分类器部分
        # ========================================
        
        # F6: 全连接层
        # 输入: 120 → 输出: 84
        self.fc1 = nn.Linear(120, 84)
        
        # 输出层
        # 输入: 84 → 输出: num_classes
        self.fc2 = nn.Linear(84, num_classes)
    
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: (batch, 1, H, W) 输入图像
               如果 H, W = 28（MNIST），会自动 padding
        
        返回:
            logits: (batch, num_classes)
        """
        # 如果输入是 28×28，padding 到 32×32
        if x.size(2) == 28:
            x = F.pad(x, (2, 2, 2, 2))  # 左右上下各 padding 2
        
        # C1 + S2
        x = self.conv1(x)           # (batch, 6, 28, 28)
        x = self.activation(x)
        x = self.pool1(x)           # (batch, 6, 14, 14)
        
        # C3 + S4
        x = self.conv2(x)           # (batch, 16, 10, 10)
        x = self.activation(x)
        x = self.pool2(x)           # (batch, 16, 5, 5)
        
        # C5
        x = self.conv3(x)           # (batch, 120, 1, 1)
        x = self.activation(x)
        
        # Flatten
        x = x.view(x.size(0), -1)   # (batch, 120)
        
        # F6 + Output
        x = self.fc1(x)             # (batch, 84)
        x = self.activation(x)
        x = self.fc2(x)             # (batch, num_classes)
        
        return x


# 测试 LeNet-5
model = LeNet5(num_classes=10)
x = torch.randn(1, 1, 28, 28)  # MNIST 尺寸
y = model(x)

print("LeNet-5 测试：")
print(f"  输入: {x.shape}")
print(f"  输出: {y.shape}")
print(f"  参数量: {sum(p.numel() for p in model.parameters()):,}")

## 1.4 在 MNIST 上训练 LeNet-5

In [ ]:
# 加载 MNIST 数据集
print("加载 MNIST 数据集...")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 均值和标准差
])

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

print(f"训练集: {len(train_data)} 样本")
print(f"测试集: {len(test_data)} 样本")

# 可视化一些样本
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axes.flat):
    img, label = train_data[i]
    ax.imshow(img.squeeze(), cmap='gray')
    ax.set_title(f'Label: {label}')
    ax.axis('off')
plt.suptitle('MNIST 样本', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
def train_model(model, train_loader, test_loader, epochs=5, lr=0.01, device='cpu'):
    """
    训练模型
    
    返回:
        history: 包含训练历史的字典
    """
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'train_acc': [], 'test_acc': []}
    
    for epoch in range(epochs):
        # 训练
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        
        train_loss = total_loss / len(train_loader)
        train_acc = 100 * correct / total
        
        # 测试
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                correct += (out.argmax(1) == y).sum().item()
                total += y.size(0)
        
        test_acc = 100 * correct / total
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        
        print(f"Epoch {epoch+1}/{epochs}: Loss={train_loss:.4f}, Train Acc={train_acc:.2f}%, Test Acc={test_acc:.2f}%")
    
    return history


# 训练 LeNet-5
print("\n训练 LeNet-5 (原版 Tanh 激活)...")
lenet_original = LeNet5(use_original_activation=True)
history_original = train_model(lenet_original, train_loader, test_loader, epochs=5, device=device)

print(f"\n最终测试准确率: {history_original['test_acc'][-1]:.2f}%")

## 1.5 LeNet-5 小结

**核心贡献**：
1. 确立了 CNN 的基本架构模式：卷积 → 池化 → 卷积 → 池化 → 全连接
2. 参数共享：同一个卷积核在整个图像上滑动，大大减少参数量
3. 局部感受野：每个神经元只看局部区域，逐层扩大感受野

**局限性**：
- 使用 Tanh 激活，存在梯度消失问题
- 网络较浅（5 层），表达能力有限
- 计算能力限制，无法处理大尺寸图像

---

# Part 2: AlexNet (2012) - 深度学习的复兴

## 2.1 历史背景：ImageNet 与深度学习革命

2012 年，Alex Krizhevsky 等人凭借 AlexNet 在 ImageNet 大规模视觉识别挑战赛（ILSVRC）中以巨大优势夺冠：

| 年份 | 方法 | Top-5 错误率 |
|------|------|-------------|
| 2011 | 传统方法 (SIFT + FV) | 25.8% |
| **2012** | **AlexNet** | **15.3%** ← 降低 10%！|
| 2012 | 第二名 (传统方法) | 26.2% |

这一结果震惊了整个计算机视觉界，标志着深度学习时代的开始。

## 2.2 AlexNet 的核心创新

AlexNet 引入了多项关键创新：

| 创新 | 说明 | 影响 |
|------|------|------|
| **ReLU 激活** | 用 max(0, x) 替代 Tanh/Sigmoid | 训练速度快 6 倍，缓解梯度消失 |
| **Dropout** | 训练时随机丢弃神经元 | 强正则化，防止过拟合 |
| **GPU 训练** | 首次使用双 GPU 并行 | 大幅加速训练 |
| **LRN** | 局部响应归一化 | 侧向抑制（后被 BN 取代）|
| **数据增强** | 随机裁剪、翻转、颜色抖动 | 扩充数据，提升泛化 |
| **重叠池化** | stride < kernel_size | 略微提升准确率 |

In [ ]:
# 可视化 ReLU vs Sigmoid vs Tanh

x = np.linspace(-5, 5, 200)

# 各激活函数
sigmoid = 1 / (1 + np.exp(-x))
tanh = np.tanh(x)
relu = np.maximum(0, x)

# 各激活函数的导数
sigmoid_grad = sigmoid * (1 - sigmoid)
tanh_grad = 1 - tanh**2
relu_grad = (x > 0).astype(float)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 激活函数
axes[0].plot(x, sigmoid, 'b-', label='Sigmoid', linewidth=2)
axes[0].plot(x, tanh, 'g-', label='Tanh', linewidth=2)
axes[0].plot(x, relu, 'r-', label='ReLU', linewidth=2)
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('激活函数对比')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-2, 5)

# 导数
axes[1].plot(x, sigmoid_grad, 'b-', label='Sigmoid\'', linewidth=2)
axes[1].plot(x, tanh_grad, 'g-', label='Tanh\'', linewidth=2)
axes[1].plot(x, relu_grad, 'r-', label='ReLU\'', linewidth=2)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].axhline(y=1, color='gray', linestyle=':', alpha=0.5)
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('x')
axes[1].set_ylabel("f'(x)")
axes[1].set_title('激活函数导数（梯度）对比')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('为什么 ReLU 更好？', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 ReLU 的优势：")
print("   1. 导数恒为 1（x > 0 时），不会梯度消失")
print("   2. 计算简单：max(0, x)，比 exp 快得多")
print("   3. 稀疏激活：负值变 0，产生稀疏表示")

## 2.3 AlexNet 架构详解

```
输入 (3, 224, 224)  ← 彩色图像（原论文是 227×227）
        ↓
   Conv1: 11×11, 96 filters, stride=4  → (96, 55, 55)
        ↓ LRN + MaxPool 3×3
   Conv2: 5×5, 256 filters, pad=2      → (256, 27, 27)
        ↓ LRN + MaxPool 3×3
   Conv3: 3×3, 384 filters, pad=1      → (384, 13, 13)
        ↓
   Conv4: 3×3, 384 filters, pad=1      → (384, 13, 13)
        ↓
   Conv5: 3×3, 256 filters, pad=1      → (256, 13, 13)
        ↓ MaxPool 3×3
   FC6: 4096 + Dropout(0.5)
        ↓
   FC7: 4096 + Dropout(0.5)
        ↓
   Output: 1000 classes
```

**参数量**：约 6000 万（60M），其中绝大部分在全连接层

In [ ]:
class AlexNet(nn.Module):
    """
    AlexNet 实现 (Krizhevsky et al., 2012)
    
    核心创新:
        - ReLU 激活函数
        - Dropout 正则化
        - 重叠池化 (overlapping pooling)
        - LRN (可选，现已很少使用)
    
    输入:
        x: (batch, 3, 224, 224) 彩色图像
    
    输出:
        logits: (batch, num_classes)
    """
    
    def __init__(self, num_classes=1000, dropout=0.5):
        super().__init__()
        
        # ========================================
        # 特征提取部分（5 个卷积层）
        # ========================================
        self.features = nn.Sequential(
            # Conv1: 大卷积核快速降维
            # 输入: (3, 224, 224) → 输出: (96, 55, 55)
            # 计算: (224 - 11) / 4 + 1 = 54.25 → 55 (因为 padding)
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            # 重叠池化: kernel=3, stride=2（stride < kernel）
            nn.MaxPool2d(kernel_size=3, stride=2),  # → (96, 27, 27)
            
            # Conv2
            # 输入: (96, 27, 27) → 输出: (256, 27, 27)
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # → (256, 13, 13)
            
            # Conv3: 开始使用 3×3 小卷积
            # 输入: (256, 13, 13) → 输出: (384, 13, 13)
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            # Conv4
            # 输入: (384, 13, 13) → 输出: (384, 13, 13)
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            # Conv5
            # 输入: (384, 13, 13) → 输出: (256, 13, 13)
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # → (256, 6, 6)
        )
        
        # ========================================
        # 分类器部分（3 个全连接层）
        # ========================================
        self.classifier = nn.Sequential(
            # Dropout: 训练时随机丢弃 50% 神经元
            nn.Dropout(p=dropout),
            
            # FC6: 256 × 6 × 6 = 9216 → 4096
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            
            nn.Dropout(p=dropout),
            
            # FC7: 4096 → 4096
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            
            # Output: 4096 → num_classes
            nn.Linear(4096, num_classes),
        )
    
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: (batch, 3, 224, 224) 输入图像
        
        返回:
            logits: (batch, num_classes)
        """
        x = self.features(x)            # (batch, 256, 6, 6)
        x = x.view(x.size(0), -1)       # (batch, 9216)
        x = self.classifier(x)          # (batch, num_classes)
        return x


# 测试 AlexNet
alexnet = AlexNet(num_classes=1000)
x = torch.randn(1, 3, 224, 224)
y = alexnet(x)

print("AlexNet 测试：")
print(f"  输入: {x.shape}")
print(f"  输出: {y.shape}")
print(f"  参数量: {sum(p.numel() for p in alexnet.parameters()):,}")

# 分析参数分布
conv_params = sum(p.numel() for name, p in alexnet.named_parameters() if 'features' in name)
fc_params = sum(p.numel() for name, p in alexnet.named_parameters() if 'classifier' in name)
print(f"\n  卷积层参数: {conv_params:,} ({conv_params / (conv_params + fc_params) * 100:.1f}%)")
print(f"  全连接层参数: {fc_params:,} ({fc_params / (conv_params + fc_params) * 100:.1f}%)")
print("\n💡 大部分参数集中在全连接层，这是后续架构优化的重点")

## 2.4 Dropout 详解

Dropout 是 AlexNet 引入的重要正则化技术：

**训练时**：以概率 $p$ 随机将神经元输出置为 0

**测试时**：使用所有神经元，但输出乘以 $(1-p)$

（PyTorch 在训练时会自动缩放，测试时不需要额外处理）

In [ ]:
# 可视化 Dropout 效果

# 创建一个简单的 Dropout 层
dropout = nn.Dropout(p=0.5)

# 输入
x = torch.ones(1, 10)

# 训练模式
dropout.train()
outputs_train = [dropout(x).numpy().flatten() for _ in range(5)]

# 测试模式
dropout.eval()
output_eval = dropout(x).numpy().flatten()

fig, axes = plt.subplots(2, 3, figsize=(12, 6))

# 可视化训练时的 5 次前向传播
for i, (ax, out) in enumerate(zip(axes[0], outputs_train[:3])):
    colors = ['red' if v == 0 else 'green' for v in out]
    ax.bar(range(10), out * 0.5 + 0.5, color=colors)  # 缩放便于显示
    ax.set_ylim(0, 1.5)
    ax.set_title(f'训练: 第 {i+1} 次前向传播')
    ax.set_xticks(range(10))
    dropped = sum(1 for v in out if v == 0)
    ax.set_xlabel(f'丢弃 {dropped}/10 个神经元')

for i, (ax, out) in enumerate(zip(axes[1][:2], outputs_train[3:5])):
    colors = ['red' if v == 0 else 'green' for v in out]
    ax.bar(range(10), out * 0.5 + 0.5, color=colors)
    ax.set_ylim(0, 1.5)
    ax.set_title(f'训练: 第 {i+4} 次前向传播')
    ax.set_xticks(range(10))
    dropped = sum(1 for v in out if v == 0)
    ax.set_xlabel(f'丢弃 {dropped}/10 个神经元')

# 测试模式
axes[1][2].bar(range(10), output_eval, color='blue')
axes[1][2].set_ylim(0, 1.5)
axes[1][2].set_title('测试: 所有神经元激活')
axes[1][2].set_xticks(range(10))
axes[1][2].set_xlabel('无丢弃')

plt.suptitle('Dropout 工作原理', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Dropout 的作用：")
print("   1. 防止过拟合：强制网络学习更鲁棒的特征")
print("   2. 相当于训练多个子网络的集成")
print("   3. 减少神经元之间的依赖关系")

## 2.5 使用 torchvision 预训练模型

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights

# 加载预训练的 AlexNet
print("加载预训练 AlexNet...")
pretrained_alexnet = alexnet(weights=AlexNet_Weights.DEFAULT)
pretrained_alexnet.eval()

# 模型信息
print(f"\n参数量: {sum(p.numel() for p in pretrained_alexnet.parameters()):,}")

# 测试推理
x = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    output = pretrained_alexnet(x)
print(f"输出形状: {output.shape}")
print(f"预测类别: {output.argmax().item()}")

## 2.6 AlexNet 小结

**核心贡献**：
1. **ReLU**：解决梯度消失，加速训练
2. **Dropout**：强正则化，防止过拟合
3. **GPU 训练**：首次大规模使用 GPU
4. **数据增强**：提升泛化能力

**局限性**：
- 全连接层参数量巨大（约 95%）
- 第一层使用 11×11 大卷积核，计算量大
- 架构设计较为随意，缺乏系统性

---

# Part 3: VGGNet (2014) - 深度的探索

## 3.1 设计哲学

VGGNet 由牛津大学 Visual Geometry Group 提出，核心思想很简单：

> **用小卷积核（3×3）堆叠替代大卷积核，增加网络深度**

VGGNet 证明了一个重要结论：**网络深度对性能至关重要**

## 3.2 为什么 3×3 更好？

### 感受野等效

两个 3×3 卷积层的感受野 = 一个 5×5 卷积层

三个 3×3 卷积层的感受野 = 一个 7×7 卷积层

```
             感受野 = 5×5                感受野 = 5×5
        ┌─────────────────┐         ┌─────────────────┐
        │                 │         │  3×3    3×3     │
        │      5×5        │   vs    │   →  ─→        │
        │                 │         │                 │
        └─────────────────┘         └─────────────────┘
        
           单个 5×5 卷积                两个 3×3 卷积
           25 个参数                   2 × 9 = 18 个参数
```

### 优势分析

| 方案 | 参数量 | 非线性次数 |
|------|--------|------------|
| 1 × 5×5 | 25C² | 1 |
| 2 × 3×3 | 18C² | 2 |
| 1 × 7×7 | 49C² | 1 |
| 3 × 3×3 | 27C² | 3 |

（C 为通道数）

**结论**：3×3 堆叠有更少的参数和更多的非线性！

In [ ]:
# 可视化感受野等效

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

def draw_receptive_field(ax, title, layers):
    """绘制感受野示意图"""
    ax.set_xlim(-0.5, 7.5)
    ax.set_ylim(-0.5, 7.5)
    
    # 绘制网格
    for i in range(8):
        for j in range(8):
            rect = plt.Rectangle((i, j), 1, 1, fill=False, edgecolor='gray', linewidth=0.5)
            ax.add_patch(rect)
    
    # 高亮感受野区域
    colors = ['#3498db', '#e74c3c', '#2ecc71']
    for i, (size, color) in enumerate(zip(layers, colors)):
        offset = (7 - size) / 2
        rect = plt.Rectangle((offset, offset), size, size, 
                              fill=True, facecolor=color, alpha=0.3,
                              edgecolor=color, linewidth=2)
        ax.add_patch(rect)
    
    # 中心点
    ax.plot(3.5, 3.5, 'ko', markersize=10)
    
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

# 5×5 感受野
draw_receptive_field(axes[0], '单个 5×5 卷积\n参数: 25C²', [5])

# 两个 3×3 感受野
draw_receptive_field(axes[1], '两个 3×3 卷积\n参数: 18C²', [5, 3])
axes[1].text(3.5, -0.8, '第1层 → 第2层', ha='center', fontsize=10)

# 三个 3×3 感受野（等价于 7×7）
draw_receptive_field(axes[2], '三个 3×3 卷积\n参数: 27C² (vs 7×7: 49C²)', [7, 5, 3])
axes[2].text(3.5, -0.8, '第1层 → 第2层 → 第3层', ha='center', fontsize=10)

plt.suptitle('3×3 卷积堆叠的感受野等效', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 参数量对比
print("参数量对比（假设 C=64）：")
C = 64
print(f"  5×5 单层: {25 * C * C:,} 参数")
print(f"  3×3 两层: {2 * 9 * C * C:,} 参数 (节省 {1 - 18/25:.0%})")
print(f"  7×7 单层: {49 * C * C:,} 参数")
print(f"  3×3 三层: {3 * 9 * C * C:,} 参数 (节省 {1 - 27/49:.0%})")

## 3.3 VGG 架构配置

VGGNet 有多个变体，通过不同的深度配置：

| 配置 | 层数 | 说明 |
|------|------|------|
| VGG-11 | 11 | 8 conv + 3 fc |
| VGG-13 | 13 | 10 conv + 3 fc |
| **VGG-16** | 16 | 13 conv + 3 fc ← 最常用 |
| VGG-19 | 19 | 16 conv + 3 fc |

所有变体的共同特点：
- 只使用 3×3 卷积（stride=1, padding=1）
- 只使用 2×2 池化（stride=2）
- 通道数按 64 → 128 → 256 → 512 递增
- 最后 3 个全连接层结构相同

In [ ]:
# VGG 配置字典
# 数字表示卷积层通道数，'M' 表示 MaxPool

VGG_CONFIGS = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

# 可视化配置对比
fig, ax = plt.subplots(figsize=(14, 6))

configs = ['VGG11', 'VGG13', 'VGG16', 'VGG19']
colors = {'64': '#3498db', '128': '#2ecc71', '256': '#f39c12', '512': '#e74c3c', 'M': '#95a5a6'}

for i, name in enumerate(configs):
    cfg = VGG_CONFIGS[name]
    x = 0
    for item in cfg:
        if item == 'M':
            # 池化层
            rect = plt.Rectangle((x, i - 0.3), 0.3, 0.6, facecolor=colors['M'], edgecolor='black')
            ax.add_patch(rect)
            x += 0.4
        else:
            # 卷积层
            rect = plt.Rectangle((x, i - 0.3), 0.8, 0.6, facecolor=colors[str(item)], edgecolor='black')
            ax.add_patch(rect)
            ax.text(x + 0.4, i, str(item), ha='center', va='center', fontsize=8, color='white')
            x += 0.9

# 图例
legend_elements = [
    plt.Rectangle((0, 0), 1, 1, facecolor=colors['64'], label='64 channels'),
    plt.Rectangle((0, 0), 1, 1, facecolor=colors['128'], label='128 channels'),
    plt.Rectangle((0, 0), 1, 1, facecolor=colors['256'], label='256 channels'),
    plt.Rectangle((0, 0), 1, 1, facecolor=colors['512'], label='512 channels'),
    plt.Rectangle((0, 0), 1, 1, facecolor=colors['M'], label='MaxPool'),
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=9)

ax.set_yticks(range(len(configs)))
ax.set_yticklabels(configs)
ax.set_xlim(-0.5, 20)
ax.set_ylim(-1, len(configs))
ax.set_xlabel('层顺序')
ax.set_title('VGG 架构配置对比', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# 统计信息
print("各配置统计：")
for name, cfg in VGG_CONFIGS.items():
    conv_layers = sum(1 for c in cfg if c != 'M')
    pool_layers = sum(1 for c in cfg if c == 'M')
    print(f"  {name}: {conv_layers} conv + {pool_layers} pool + 3 fc = {conv_layers + pool_layers + 3} 层")

## 3.4 PyTorch 实现 VGGNet

In [ ]:
class VGG(nn.Module):
    """
    VGGNet 实现 (Simonyan & Zisserman, 2014)
    
    设计特点:
        - 全部使用 3×3 卷积（stride=1, padding=1）
        - 全部使用 2×2 池化（stride=2）
        - 通道数逐步增加: 64 → 128 → 256 → 512
        - 配置驱动，易于扩展
    
    输入:
        x: (batch, 3, 224, 224) 彩色图像
    
    输出:
        logits: (batch, num_classes)
    """
    
    def __init__(self, config_name='VGG16', num_classes=1000, dropout=0.5):
        """
        参数:
            config_name: 配置名称，可选 'VGG11', 'VGG13', 'VGG16', 'VGG19'
            num_classes: 分类数量
            dropout: Dropout 概率
        """
        super().__init__()
        
        # 获取配置
        if config_name not in VGG_CONFIGS:
            raise ValueError(f"Unknown config: {config_name}")
        config = VGG_CONFIGS[config_name]
        
        # ========================================
        # 特征提取部分（卷积层 + 池化层）
        # ========================================
        self.features = self._make_layers(config)
        
        # ========================================
        # 分类器部分（3 个全连接层）
        # ========================================
        self.classifier = nn.Sequential(
            # FC1: 512 × 7 × 7 = 25088 → 4096
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            
            # FC2: 4096 → 4096
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            
            # Output: 4096 → num_classes
            nn.Linear(4096, num_classes),
        )
        
        # 权重初始化
        self._init_weights()
    
    def _make_layers(self, config):
        """
        根据配置构建卷积层
        
        参数:
            config: 层配置列表，如 [64, 64, 'M', 128, ...]
        
        返回:
            nn.Sequential: 卷积层序列
        """
        layers = []
        in_channels = 3  # RGB 输入
        
        for item in config:
            if item == 'M':
                # MaxPool 2×2, stride=2
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                # Conv 3×3 + ReLU
                out_channels = item
                layers.extend([
                    nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                    nn.BatchNorm2d(out_channels),  # 现代版本添加 BN
                    nn.ReLU(inplace=True),
                ])
                in_channels = out_channels
        
        return nn.Sequential(*layers)
    
    def _init_weights(self):
        """权重初始化"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: (batch, 3, 224, 224) 输入图像
        
        返回:
            logits: (batch, num_classes)
        """
        x = self.features(x)        # (batch, 512, 7, 7)
        x = x.view(x.size(0), -1)   # (batch, 25088)
        x = self.classifier(x)      # (batch, num_classes)
        return x


# 测试 VGG-16
vgg16 = VGG(config_name='VGG16', num_classes=1000)
x = torch.randn(1, 3, 224, 224)
y = vgg16(x)

print("VGG-16 测试：")
print(f"  输入: {x.shape}")
print(f"  输出: {y.shape}")
print(f"  参数量: {sum(p.numel() for p in vgg16.parameters()):,}")

# 测试其他配置
print("\n各配置参数量：")
for name in ['VGG11', 'VGG13', 'VGG16', 'VGG19']:
    model = VGG(config_name=name, num_classes=1000)
    params = sum(p.numel() for p in model.parameters())
    print(f"  {name}: {params:,} ({params/1e6:.1f}M)")

## 3.5 VGGNet 小结

**核心贡献**：
1. 证明了 **深度** 对于网络性能的重要性
2. 确立了 **3×3 小卷积** 的设计范式
3. 提供了清晰、规整的架构设计模板

**局限性**：
- 参数量巨大（约 138M），训练和部署都很困难
- 全连接层占据绝大部分参数
- 深度增加到一定程度后效果不再提升（退化问题）→ 引出 ResNet

---

# Part 4: 三者对比实验

在 CIFAR-10 上对比 LeNet、AlexNet（简化版）、VGG（简化版）的性能。

In [ ]:
# 为 CIFAR-10 设计简化版本

class LeNetForCIFAR(nn.Module):
    """LeNet 改编版，适配 CIFAR-10 (32×32 RGB)"""
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


class AlexNetForCIFAR(nn.Module):
    """AlexNet 简化版，适配 CIFAR-10"""
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 192, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


class VGGForCIFAR(nn.Module):
    """VGG 简化版，适配 CIFAR-10"""
    
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 4
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2 * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


# 统计参数量
models_dict = {
    'LeNet': LeNetForCIFAR(),
    'AlexNet': AlexNetForCIFAR(),
    'VGG': VGGForCIFAR(),
}

print("模型参数量对比：")
for name, model in models_dict.items():
    params = sum(p.numel() for p in model.parameters())
    print(f"  {name}: {params:,}")

In [ ]:
# 加载 CIFAR-10
print("加载 CIFAR-10 数据集...")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

cifar_train = datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
cifar_test = datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)

cifar_train_loader = DataLoader(cifar_train, batch_size=128, shuffle=True, num_workers=2)
cifar_test_loader = DataLoader(cifar_test, batch_size=128, shuffle=False, num_workers=2)

print(f"训练集: {len(cifar_train)} 样本")
print(f"测试集: {len(cifar_test)} 样本")

In [ ]:
def train_and_evaluate(model, train_loader, test_loader, epochs=10, lr=0.001):
    """训练并评估模型"""
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    history = {'train_acc': [], 'test_acc': [], 'time': []}
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # 训练
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)
        train_acc = 100 * correct / total
        
        # 测试
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                correct += (out.argmax(1) == y).sum().item()
                total += y.size(0)
        test_acc = 100 * correct / total
        
        epoch_time = time.time() - start_time
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        history['time'].append(epoch_time)
        
        print(f"  Epoch {epoch+1}/{epochs}: Train={train_acc:.1f}%, Test={test_acc:.1f}%, Time={epoch_time:.1f}s")
        scheduler.step()
    
    return history


# 训练对比
EPOCHS = 10
results = {}

for name in ['LeNet', 'AlexNet', 'VGG']:
    print(f"\n{'='*50}")
    print(f"训练 {name}")
    print(f"{'='*50}")
    
    # 重新创建模型
    if name == 'LeNet':
        model = LeNetForCIFAR()
    elif name == 'AlexNet':
        model = AlexNetForCIFAR()
    else:
        model = VGGForCIFAR()
    
    results[name] = train_and_evaluate(model, cifar_train_loader, cifar_test_loader, epochs=EPOCHS)

In [ ]:
# 可视化对比结果

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

colors = {'LeNet': '#2ecc71', 'AlexNet': '#e74c3c', 'VGG': '#3498db'}

# 1. 测试准确率曲线
for name, history in results.items():
    axes[0].plot(range(1, EPOCHS+1), history['test_acc'], 
                 label=name, color=colors[name], linewidth=2, marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy (%)')
axes[0].set_title('测试准确率曲线')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. 最终准确率对比
names = list(results.keys())
final_acc = [results[n]['test_acc'][-1] for n in names]
bars = axes[1].bar(names, final_acc, color=[colors[n] for n in names])
axes[1].set_ylabel('Test Accuracy (%)')
axes[1].set_title('最终测试准确率')
axes[1].set_ylim(50, 95)
for bar, acc in zip(bars, final_acc):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'{acc:.1f}%', ha='center', fontsize=11)

# 3. 参数量对比
params = {
    'LeNet': sum(p.numel() for p in LeNetForCIFAR().parameters()),
    'AlexNet': sum(p.numel() for p in AlexNetForCIFAR().parameters()),
    'VGG': sum(p.numel() for p in VGGForCIFAR().parameters()),
}
param_values = [params[n] / 1e6 for n in names]
bars = axes[2].bar(names, param_values, color=[colors[n] for n in names])
axes[2].set_ylabel('Parameters (M)')
axes[2].set_title('模型参数量')
for bar, p in zip(bars, param_values):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                 f'{p:.2f}M', ha='center', fontsize=11)

plt.suptitle('LeNet vs AlexNet vs VGG 对比 (CIFAR-10)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 总结表格
print("\n" + "="*60)
print("实验总结")
print("="*60)
print(f"\n{'模型':<10} {'参数量':>12} {'最终准确率':>12} {'平均Epoch时间':>14}")
print("-" * 50)
for name in names:
    p = params[name]
    acc = results[name]['test_acc'][-1]
    t = np.mean(results[name]['time'])
    print(f"{name:<10} {p/1e6:>10.2f}M {acc:>11.1f}% {t:>13.1f}s")

---

## 📝 本章小结

### 三个架构的核心贡献

| 架构 | 年份 | 核心贡献 | 历史意义 |
|------|------|----------|----------|
| **LeNet-5** | 1998 | 卷积 + 池化 + 全连接 | 确立 CNN 基本范式 |
| **AlexNet** | 2012 | ReLU + Dropout + GPU | 开启深度学习时代 |
| **VGGNet** | 2014 | 3×3 小卷积堆叠 | 证明深度的重要性 |

### 发展脉络

```
LeNet-5 (1998)           AlexNet (2012)           VGGNet (2014)
    │                        │                        │
    ├─ 卷积 + 池化           ├─ ReLU 激活              ├─ 3×3 小卷积
    ├─ Tanh 激活             ├─ Dropout 正则化         ├─ 更深的网络
    └─ 手写识别              ├─ GPU 训练               └─ 规整的设计
                             └─ ImageNet 冠军
```

### 关键代码模式

```python
# LeNet 模式：卷积 → 激活 → 池化
nn.Conv2d(1, 6, 5), nn.Tanh(), nn.AvgPool2d(2)

# AlexNet 模式：加入 Dropout
nn.Conv2d(3, 96, 11, stride=4), nn.ReLU(), nn.Dropout(0.5)

# VGG 模式：3×3 卷积堆叠
nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU()
```

---

## 🎯 练习题

### 练习 1：修改 LeNet 适配彩色图像
**难度**：⭐

修改 LeNet5 类，使其能够处理 3 通道的彩色图像（如 CIFAR-10）。

### 练习 2：分析 VGG 中 3×3 堆叠的感受野
**难度**：⭐⭐

计算 VGG-16 中每个池化层之后的感受野大小。提示：考虑 stride 和 kernel size。

### 练习 3：实现 AlexNet 的 LRN 层
**难度**：⭐⭐⭐

LRN（Local Response Normalization）公式：
$$b_{x,y}^i = a_{x,y}^i / \left(k + \alpha \sum_{j=\max(0, i-n/2)}^{\min(N-1, i+n/2)} (a_{x,y}^j)^2 \right)^\beta$$

用 PyTorch 实现这个归一化层。

---

## 🚀 下一步

继续学习 **07b_inception_densenet.ipynb** - Inception 与 DenseNet

探索更现代的架构创新：多尺度特征提取和密集连接。